In [7]:
### implementing sample data using ready made package ######
import elm
data = np.loadtxt("C:\Users\Shilpa\Desktop\iris.data.txt")
# create a classifier
elmk = elm.ELMKernel()

# search for best parameter for this dataset
# define "kfold" cross-validation method, "accuracy" as a objective function
# to be optimized and perform 10 searching steps.
# best parameters will be saved inside 'elmk' object
elmk.search_param(data, cv="kfold", of="accuracy", eval=10)

# split data in training and testing sets
# use 80% of dataset to training and shuffle data before splitting
tr_set, te_set = elm.split_sets(data, training_percent=.8, perm=True)

#train and test
# results are Error objects
tr_result = elmk.train(tr_set)
te_result = elmk.test(te_set)

print(te_result.get_accuracy)

elmk
##### Start search #####
Kernel function:  rbf  best cv value:  0.966666666667
Kernel function:  linear  best cv value:  0.973333333333
Kernel function:  poly  best cv value:  0.986666666667
##### Search complete #####

Regressor Parameters

Regularization coefficient:  1.42206186347
Kernel Function:  poly
Kernel parameters:  [2.176470868723547, 3.0247715958014876]

CV error:  0.986666666667

<bound method Error.get_accuracy of <elm.mltools.Error object at 0x0000000013FD9390>>


In [1]:
###############Extreme Learning Machine, is an easy way of training Artificial Neural Networks with a hidden layer###########


import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

class ELMRegressor():
    def __init__(self, n_hidden_units):
        self.n_hidden_units = n_hidden_units

    def fit(self, X, labels):
        X = np.column_stack([X, np.ones([X.shape[0], 1])])
        self.random_weights = np.random.randn(X.shape[1], self.n_hidden_units)
        G = np.tanh(X.dot(self.random_weights))
        self.w_elm = np.linalg.pinv(G).dot(labels)

    def predict(self, X):
        X = np.column_stack([X, np.ones([X.shape[0], 1])])
        G = np.tanh(X.dot(self.random_weights))
        return G.dot(self.w_elm)
    
test_maes_dictionary = dict()

plt.style.use('ggplot')
sns.set_context("talk")
np.random.seed(0)

## DATA PREPROCESSING
X, y = load_diabetes().values()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

stdScaler_data = StandardScaler()
X_train = stdScaler_data.fit_transform(X_train)
X_test = stdScaler_data.transform(X_test)

stdScaler_target = StandardScaler()
y_train = stdScaler_target.fit_transform(y_train)  # /max(y_train)
y_test = stdScaler_target.transform(y_test)  # /max(y_train)
max_y_train = max(abs(y_train))
y_train = y_train / max_y_train
y_test = y_test / max_y_train

## ELM TRAINING
MAE_TRAIN_MINS = []
MAE_TEST_MINS = []

for M in range(1, 200, 1):
    MAES_TRAIN = []
    MAES_TEST = []
    # print "Training with %s neurons..."%M
    for i in range(30):
        ELM = ELMRegressor(M)
        ELM.fit(X_train, y_train)
        prediction = ELM.predict(X_train)
        MAES_TRAIN.append(mean_absolute_error(y_train,
                                              prediction))

        prediction = ELM.predict(X_test)
        MAES_TEST.append(mean_absolute_error(y_test,
                                             prediction))
    MAE_TEST_MINS.append(min(MAES_TEST))
    MAE_TRAIN_MINS.append(MAES_TRAIN[np.argmin(MAES_TEST)])

print "Minimum MAE ELM =", min(MAE_TEST_MINS)
test_maes_dictionary["ELM"] = min(MAE_TEST_MINS)

## LINEAR REGRESSION TRAINING
mae = []
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X_test)
mae.append(mean_absolute_error(y_test, prediction))
print "Minimum MAE LR =", min(mae)
test_maes_dictionary["LinReg"] = min(mae)

## K-NEAREST NEIGHBORS TRAINING
mae = []
for N in range(1, 51):
    kn = KNeighborsRegressor()
    kn.fit(X_train, y_train)
    prediction = kn.predict(X_test)
    mae.append(mean_absolute_error(y_test, prediction))
print "Minimum MAE KNN =", min(mae)
test_maes_dictionary["KNN"] = min(mae)

## DECISION TREES TRAINING
mae = []
for max_depth in range(1, 51):
    for min_samples_split in range(1, 102, 5):
        tree = DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_samples_split)
        tree.fit(X_train, y_train)
        prediction = tree.predict(X_test)
        mae.append(mean_absolute_error(y_test, prediction))
print "Minimum MAE TREE = ", min(mae)
test_maes_dictionary["Dec. Tree"] = min(mae)

## SUPPORT VECTORS MACHINE TRAINING
mae = []
for kernel in ["rbf", "linear", "poly", "sigmoid"]:
    svr = SVR(kernel=kernel)
    svr.fit(X_train, y_train)
    prediction = svr.predict(X_test)
    mae.append(mean_absolute_error(y_test, prediction))
print "Minimum MAE SVR = ", min(mae)
test_maes_dictionary["SVM"] = min(mae)

## RANDOM FOREST TRAINING
mae = []
for n_estimators in range(10, 1100, 100):
    rf = RandomForestRegressor(n_estimators=n_estimators)
    rf.fit(X_train, y_train)
    prediction = rf.predict(X_test)
    mae.append(mean_absolute_error(y_test, prediction))
print "Minimum MAE R.Forest = ", min(mae)
test_maes_dictionary["R. Forest"] = min(mae)

#############################################################################################
## PLOTTING THE RESULTS
df = pd.DataFrame()
df["test"] = MAE_TEST_MINS
df["train"] = MAE_TRAIN_MINS
ax = df.plot()
ax.set_xlabel("Number of Neurons in the hidden layer")
ax.set_ylabel("Mean Absolute Error")
ax.set_title(
    "Extreme Learning Machine error obtained for the Diabetes dataset \n when varying the number of neurons in the "
    "hidden layer (min. at 23 neurons)")
plt.show()

D = test_maes_dictionary
plt.bar(range(len(D)), D.values(), align='center')
plt.xticks(range(len(D)), D.keys())
plt.ylabel("Mean Absolute Error")
plt.title("Error Comparison between Classic Regression Models and ELM")
plt.show()   

Minimum MAE ELM = 0.231925412851
Minimum MAE LR = 0.239039348749
Minimum MAE KNN = 0.27152636119
Minimum MAE TREE =  0.25902477534
Minimum MAE SVR =  0.241853736229
Minimum MAE R.Forest =  0.257440529652


In [ ]:

##########Implementing ELM for SLFN - code implemention, Couldnt complete the code by the deadline################


import numpy as np
from scipy.linalg import pinv2
from abc import ABCMeta, abstractmethod
from sklearn.utils import as_float_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.preprocessing import LabelBinarizer
import unicode_literals


data = np.loadtxt("C:\Users\Shilpa\Desktop\principal_componenets_data.txt")
def GenELMRegressor(input_vector, output_values):
    """
    ELMRegressor is a regressor based on the Extreme Learning Machine.
    An Extreme Learning Machine (ELM) is a single layer feedforward
    network with a random hidden layer components and ordinary linear
    least squares fitting of the hidden->output weights by default.
    """
    node_values = self._forward_propagate(input_vector)
    deltas = self._get_betas(node_values, output_values)
    self._adjust_weights(node_values, deltas)
    known_values = dict(zip(self.network_inputs, input_vector))
    for layer in self.hidden_layers+[self.network_outputs]:
        for unit in layer:
            known_values[unit] = self._calculate_node_value(unit, known_values)
        return known_values

def get_betas(self, node_values, output_values):
    deltas = dict([node_values[o] - output_values[i] for i,o in enumerate(self.network_outputs)])
    for hidden_layer in reversed(self.hidden_layers):
        for unit in hidden_layer:
            deltas[unit] = self._calculate_delta(unit, deltas, node_values)
    return deltas

def calculate_beta(self, node, deltas, node_values):
    summed_inputs = self._get_neuron_summed_inputs(node, node_values)
    summed_weighted_deltas = sum([self.get_edge_weight(node, o)*deltas[o] for o in self.network.successors(node)])
    return self._logistic_derivative(summed_inputs) * summed_weighted_deltas

def adjust_weights(self, node_values, deltas):
    for n1, n2 in self.network.edges():
        w = self.get_edge_weight(n1, n2)
        self.set_edge_weight(n1, n2, w - (node_values[n1]*deltas[n2]))
def fit(self, X, y):
    num_features = X.shape[0]
    input_layer_size = X.shape[1]
    num_labels = len(set(y))
    theta1_0 = self.rand_init(input_layer_size, self.hidden_layer_size)
    theta2_0 = self.rand_init(self.hidden_layer_size, num_labels)
    thetas0 = self.pack_thetas(theta1_0, theta2_0)
    self.t1, self.t2 = self.unpack_thetas(_res.x, input_layer_size, self.hidden_layer_size, num_labels)

def predict(self, X):
    return self.predict_proba(X).argmax(0)

        
def randomLayerCalulation(train_set, kernel_type ):
    number_training_patterns = training_patterns.shape[0]

    if kernel_type == "rbf":
        if test_patterns is None:
            temp_omega = np.dot(
                np.sum(training_patterns ** 2, axis=1).reshape(-1, 1),
                np.ones((1, number_training_patterns)))

            temp_omega = temp_omega + temp_omega.conj().T

            omega = np.exp(
                -(2 ** kernel_param[0]) * (temp_omega - 2 * (np.dot(
                    training_patterns, training_patterns.conj().T))))

        else:
            number_test_patterns = test_patterns.shape[0]

            temp1 = np.dot(
                np.sum(training_patterns ** 2, axis=1).reshape(-1, 1),
                np.ones((1, number_test_patterns)))
            temp2 = np.dot(
                np.sum(test_patterns ** 2, axis=1).reshape(-1, 1),
                np.ones((1, number_training_patterns)))
            temp_omega = temp1 + temp2.conj().T

            omega = np.exp(- (2 ** kernel_param[0]) *
                       (temp_omega - 2 * np.dot(training_patterns,
                                                test_patterns.conj().T)))
    elif kernel_type == "linear":
        if test_patterns is None:
            omega = np.dot(training_patterns, training_patterns.conj().T)
        else:
            omega = np.dot(training_patterns, test_patterns.conj().T)

    elif kernel_type == "poly":
        # Power a**x is undefined when x is real and 'a' is negative,
        # so is necessary to force an integer value
        kernel_param[1] = round(kernel_param[1])

        if test_patterns is None:
            temp = np.dot(training_patterns, training_patterns.conj().T)+ kernel_param[0]

            omega = temp ** kernel_param[1]
        else:
            temp = np.dot(training_patterns, test_patterns.conj().T)+ kernel_param[0]
            omega = temp ** kernel_param[1]
    else:
        print("Error: Invalid or unavailable kernel function.")
        return

    return omega

def _local_train(self, training_patterns, training_expected_targets,
                 params):

    # If params not provided, uses initialized parameters values
    if not params:
        pass
    else:
        self.param_kernel_function = params[0]
        self.param_c = params[1]
        self.param_kernel_params = params[2]

    # Need to save all training patterns to perform kernel calculation at
    # testing and prediction phase
    self.training_patterns = training_patterns

    number_training_patterns = self.training_patterns.shape[0]

    # Training phase

    omega_train = self._kernel_matrix(self.training_patterns,
                                       self.param_kernel_function,
                                       self.param_kernel_params)

    self.output_weight = np.linalg.solve(
        (omega_train + np.eye(number_training_patterns) /(2 ** self.param_c)), training_expected_targets).reshape(-1, 1)
    training_predicted_targets = np.dot(omega_train, self.output_weight)
    return training_predicted_targets

def local_test(self, testing_patterns, testing_expected_targets,
                predicting):

    omega_test = self._kernel_matrix(self.training_patterns,
                                      self.param_kernel_function,
                                      self.param_kernel_params,
                                      testing_patterns)

    testing_predicted_targets = np.dot(omega_test.conj().T,
                                       self.output_weight)

    return testing_predicted_targets


    
        
        
